In [4]:
import pandas as pd

citibike_data = None

for month in range(1,13):
    if month < 10:
        month_str = f'0{month}'
    else:
        month_str = f'{month}'
    print('Loading month', month_str)
    current_month = pd.read_csv(f'../dataset/2018{month_str}-citibike-tripdata.csv')
    
    if citibike_data is None:
        citibike_data = current_month
    else:
        citibike_data = citibike_data.append(current_month)

Loading month 01
Loading month 02
Loading month 03
Loading month 04
Loading month 05
Loading month 06
Loading month 07
Loading month 08
Loading month 09
Loading month 10
Loading month 11
Loading month 12


In [5]:
citibike_data = citibike_data.dropna()

import numpy as np

citibike_data['starttime'] = pd.to_datetime(citibike_data['starttime'])
citibike_data['stoptime'] = pd.to_datetime(citibike_data['stoptime'])

citibike_data['tripduration'] = (citibike_data['stoptime'] - citibike_data['starttime']).dt.total_seconds().apply(np.floor).astype(int)

filter_columns = [
    'tripduration', 
    'starttime', 
    'stoptime', 
    'start station id',
    'start station latitude',
    'start station longitude',
    'end station id',
    'end station latitude',
    'end station longitude',
    'usertype',
    'birth year'
]
citibike_data = citibike_data[filter_columns]
citibike_data = citibike_data[citibike_data['tripduration'] <= 20*24*60*60]

citibike_data = citibike_data[citibike_data['end station latitude'] <= 41.13]
citibike_data = citibike_data[citibike_data['start station latitude'] <= 41.13]
citibike_data = citibike_data[citibike_data['end station latitude'] >= 40.12]
citibike_data = citibike_data[citibike_data['start station latitude'] >= 40.12]
citibike_data = citibike_data[citibike_data['end station longitude'] >= -74.40]
citibike_data = citibike_data[citibike_data['start station longitude'] >= -74.40]
citibike_data = citibike_data[citibike_data['end station longitude'] <= -73.19]
citibike_data = citibike_data[citibike_data['start station longitude'] <= -73.19]

citibike_data['start_weekday'] = citibike_data['starttime'].dt.day_name()

citibike_data

,tripduration,starttime,stoptime,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,usertype,birth year,start_weekday
0,970,2018-01-01 13:50:57.434,2018-01-01 14:07:08.186,72.0,40.767272,-73.993929,505.0,40.749013,-73.988484,Subscriber,1992,Monday
1,723,2018-01-01 15:33:30.182,2018-01-01 15:45:33.341,72.0,40.767272,-73.993929,3255.0,40.750585,-73.994685,Subscriber,1969,Monday
2,496,2018-01-01 15:39:18.337,2018-01-01 15:47:35.172,72.0,40.767272,-73.993929,525.0,40.755942,-74.002116,Subscriber,1956,Monday
3,306,2018-01-01 15:40:13.372,2018-01-01 15:45:20.191,72.0,40.767272,-73.993929,447.0,40.763707,-73.985162,Subscriber,1974,Monday
4,306,2018-01-01 18:14:51.568,2018-01-01 18:19:57.642,72.0,40.767272,-73.993929,3356.0,40.774667,-73.984706,Subscriber,1992,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...
1016500,734,2018-12-31 23:55:06.991,2019-01-01 00:07:21.230,336.0,40.730477,-73.999061,379.0,40.749156,-73.991600,Subscriber,1986,Monday
1016501,2551,2018-12-31 23:55:44.307,2019-01-01 00:38:15.782,3320.0,40.794067,-73.962868,2006.0,40.765909,-73.976342,Customer,1969,Monday
1016502,2376,2018-12-31 23:56:52.850,2019-01-01 00:36:29.286,427.0,40.701907,-74.013942,259.0,40.701221,-74.012342,Subscriber,1979,Monday
1016503,2758,2018-12-31 23:58:29.212,2019-01-01 00:44:27.517,3320.0,40.794067,-73.962868,281.0,40.764397,-73.973715,Customer,1969,Monday


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [13]:
features_df = citibike_data
feature_columns = [
    'tripduration', 
    'start station latitude',
    'start station longitude',
    'end station latitude',
    'end station longitude',
    'birth year',
    'start_weekday'
]
label_column = 'usertype'

features_df = features_df[feature_columns + [label_column]]
features_df

,tripduration,start station latitude,start station longitude,end station latitude,end station longitude,birth year,start_weekday,usertype
0,970,40.767272,-73.993929,40.749013,-73.988484,1992,Monday,Subscriber
1,723,40.767272,-73.993929,40.750585,-73.994685,1969,Monday,Subscriber
2,496,40.767272,-73.993929,40.755942,-74.002116,1956,Monday,Subscriber
3,306,40.767272,-73.993929,40.763707,-73.985162,1974,Monday,Subscriber
4,306,40.767272,-73.993929,40.774667,-73.984706,1992,Monday,Subscriber
...,...,...,...,...,...,...,...,...
1016500,734,40.730477,-73.999061,40.749156,-73.991600,1986,Monday,Subscriber
1016501,2551,40.794067,-73.962868,40.765909,-73.976342,1969,Monday,Customer
1016502,2376,40.701907,-74.013942,40.701221,-74.012342,1979,Monday,Subscriber
1016503,2758,40.794067,-73.962868,40.764397,-73.973715,1969,Monday,Customer


In [14]:
from sklearn import preprocessing

unique_labels = features_df['usertype'].unique()
unique_weekdays = features_df['start_weekday'].unique()
le_ut = preprocessing.LabelEncoder()
le_ut.fit(unique_labels)
le_wd = preprocessing.LabelEncoder()
le_wd.fit(unique_weekdays)

features_df['usertype'] = le_ut.transform(features_df['usertype']) 
features_df['start_weekday'] = le_wd.transform(features_df['start_weekday']) 

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [15]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(
    features_df, 
    test_size=0.33, 
    random_state=42, 
    stratify=features_df[label_column]
)

In [16]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(train_set[feature_columns], train_set[label_column])

print(clf.feature_importances_)

[0.38899479 0.06037732 0.03093029 0.02742577 0.00765714 0.34499366
 0.13962102]


In [18]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

predicted_labels = clf.predict(test_set[feature_columns])
true_labels = test_set[label_column].tolist()

print(f1_score(true_labels, predicted_labels))

print(le_ut.transform(unique_labels))
print(unique_labels)

print(classification_report(true_labels, predicted_labels))

0.9417271525023512
[1 0]
['Subscriber' 'Customer']


/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00    637643
           1       0.89      1.00      0.94   5152363

    accuracy                           0.89   5790006
   macro avg       0.44      0.50      0.47   5790006
weighted avg       0.79      0.89      0.84   5790006



In [19]:
clf2 = RandomForestClassifier(max_depth=10, random_state=0)
clf2.fit(train_set[feature_columns], train_set[label_column])

print(clf2.feature_importances_)

[0.23686432 0.02243247 0.02026253 0.02237702 0.01919279 0.63573188
 0.04313899]


In [20]:
predicted_labels = clf.predict(test_set[feature_columns])
true_labels = test_set[label_column].tolist()

print(f1_score(true_labels, predicted_labels))

print(le_ut.transform(unique_labels))
print(unique_labels)

print(classification_report(true_labels, predicted_labels))

0.9417271525023512
[1 0]
['Subscriber' 'Customer']


/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00    637643
           1       0.89      1.00      0.94   5152363

    accuracy                           0.89   5790006
   macro avg       0.44      0.50      0.47   5790006
weighted avg       0.79      0.89      0.84   5790006

